In [6]:
import requests
import pandas as pd

# Define the API endpoint
url = "https://api.artic.edu/api/v1/artworks"

# Make a GET request to the API
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the JSON data from the response
    data = response.json()
    
    # Extract relevant information from the response and create a DataFrame
    artworks = []
    for artwork in data['data']:
        # Split artist information based on newline character
        artist_info = artwork['artist_display'].split('\n')
        
        # Extract artist name and origin (if available)
        artist_name = artist_info[0].strip()  # Remove leading/trailing whitespace
        artist_origin = artist_info[1].strip() if len(artist_info) > 1 else None
        
        artwork_info = {
            'title': artwork['title'],
            'artist': artist_name,
            'artist_origin': artist_origin,  # New column for artist origin
            'medium': artwork['medium_display'],
            'date': artwork['date_display']
            # Add more fields as needed
        }
        artworks.append(artwork_info)
    
    # Convert the list of dictionaries into a DataFrame
    df = pd.DataFrame(artworks)
else:
    print("Error:", response.status_code)

df


,title,artist,artist_origin,medium,date
0,Untitled,Béla Kalman,"American, born Hungary, 1921–2011",Gelatin silver print,1935/68
1,Solarization,György Kepes,"American, born Hungary, 1906–2001",Gelatin silver print,c. 1938
2,"Lucia Moholy on a Balcony, Ascona",László Moholy-Nagy,"American, born Hungary, 1895–1946",Gelatin silver print,c. 1926
3,Composition,Bruce Alonzo Goff,"American, 1904–1982","Graphite, watercolor, tempera, ink, and silver...",n.d.
4,Joachim Schmid Reading 6.51: Skepticism Is Not...,Arthur Ou,"Taiwanese, born 1974",Gelatin silver print,"2014, printed 2019"
5,Phil Chang Reading 5.632: The Subject Does Not...,Arthur Ou,"Taiwanese, born 1974",Gelatin silver print,"2015, printed 2019"
6,Eileen Quinlan Reading 5.61: Logic Pervades th...,Arthur Ou,"Taiwanese, born 1974",Gelatin silver print,"2015, printed 2019"
7,Uta Barth Reading 4.015: The Possibility of Al...,Arthur Ou,"Taiwanese, born 1974",Gelatin silver print,"2015, printed 2019"
8,"The Brown Sisters, Brookline, Massachusetts",Nicholas Nixon,"American, born 1947",Gelatin silver print,2018
9,"The Brown Sisters, Brookline, Massachusetts",Nicholas Nixon,"American, born 1947",Gelatin silver print,"2020, printed 2020"
